In [187]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
nlp=spacy.load("en_core_web_lg")
pd.set_option("display.max_rows",None)
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\25bak\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [188]:
train=pd.read_csv("Data/train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [189]:
test=pd.read_csv("Data/test.csv")
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [190]:
print("Null values in training set")
print(train.isnull().sum())
print("Null values in testing set")
print(test.isnull().sum())

Null values in training set
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
Null values in testing set
id             0
keyword       26
location    1105
text           0
dtype: int64


In [191]:
print(train[["keyword","text"]].sample(10))

            keyword                                               text
739        bleeding  lets see how good you are at soccer when you'r...
2336     demolition  This sale and demolition trend near Metrotown ...
6075       sinkhole  Some Evansville residents told to limit water ...
4026  forest%20fire  Ashenforest floorburnt manzanita &amp; timber ...
3537     eyewitness  Freed #BokoHaram captives speak up: ÛÏI will ...
3476      explosion  ...don't think I've ever been this close to a ...
1113         bombed  @ChristophersZen @HunterLove1995 @tblack yeah ...
4433        hostage  whO'S THAT SHADOW HOLDIN ME HOSTAGE I'VE BEEN ...
5606          razed  The Latest: More Homes Razed by Northern Calif...
5915       screamed  someone's gonna get screamed at for getting th...


In [192]:
train_real=train[train["target"]==1]
train_fake=train[train["target"]==0]

In [193]:
print(train_real.shape," is the size of real train dataset")
print(train_fake.shape," is the size of fake train dataset")

(3271, 5)  is the size of real train dataset
(4342, 5)  is the size of fake train dataset


In [194]:
print(train_real["keyword"].nunique(),train_fake["keyword"].nunique())
print(test["keyword"].nunique())
print(set(train.keyword.unique())-set(test.keyword.unique()))

220 218
221
set()


In [195]:
impWords=train["keyword"].unique()[:15]
print(impWords)

[nan 'ablaze' 'accident' 'aftershock' 'airplane%20accident' 'ambulance'
 'annihilated' 'annihilation' 'apocalypse' 'armageddon' 'army' 'arson'
 'arsonist' 'attack' 'attacked']


In [196]:
# plt.figure(figsize=(15,10))
# sns.countplot(x=train["keyword"],hue=train["target"])
# plt.xticks(rotation=90)

In [197]:
unique=train["keyword"].unique().astype("str")
unique.tolist()
# uniqueCollection=[unique[i:i+15] 
#                   for i in range(0,len(unique),15)]
# plt.figure(figsize=(10,15))
# subplot_counter=1
# for group in uniqueCollection:
#     plt.figure(figsize=(10,15))
#     plt.subplot(len(uniqueCollection),1,subplot_counter)
#     countplot=sns.countplot(data=train[train["keyword"].isin(group)],x="keyword",hue="target")
#     plt.xticks(rotation=90)
#     plt.legend(title="target",loc="lower left")

['nan',
 'ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew%20up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown%20up',
 'body%20bag',
 'body%20bagging',
 'body%20bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge%20collapse',
 'buildings%20burning',
 'buildings%20on%20fire',
 'burned',
 'burning',
 'burning%20buildings',
 'bush%20fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical%20emergency',
 'cliff%20fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolat

In [198]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment=SentimentIntensityAnalyzer()

In [199]:
def lastKeyIn(info):
    keyword=info[0]
    text=info[1]
    if pd.notna(keyword):
         return keyword
    doc=nlp(text)
    high_neg=0
    negative_word=None
    for word in doc:
        if word.is_stop or word.is_oov or word.is_space or word.is_punct:
             pass
        elif word.is_ascii:
            if word.text in unique or word.lemma_ in unique:
                return word
            neg=sentiment.polarity_scores(word.text)["neg"]
            if neg>high_neg:
                  high_neg=neg
                  negative_word=word
    if high_neg<0.5:
         return("Others")
    return negative_word.lower

In [200]:
print(train.isnull().sum())
train.keyword=train[["keyword","text"]].apply(lastKeyIn,axis=1)
print(train.isnull().sum())

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


id             0
keyword        0
location    2533
text           0
target         0
dtype: int64


In [201]:
keyword=train.keyword.unique().astype("str")
keyword=keyword.tolist()
keyword=set(keyword)
keyword

{'10595077177392105327',
 '1228720749737493560',
 '13773067740709375554',
 '14012806951969610844',
 '1523267725910686442',
 '7886921362353864828',
 '920492170430266568',
 'CYCLONE',
 'Damage',
 'Earthquake',
 'Evacuation',
 'FIRE',
 'Others',
 'Seismic',
 'Thunderstorm',
 'ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew%20up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown%20up',
 'body%20bag',
 'body%20bagging',
 'body%20bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge%20collapse',
 'buildings%20burning',
 'buildings%20on%20fire',
 'burned',
 'burning',
 'burning%20buildings',
 'bush%20fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical%20emergency',
 'cliff%20fall',
 'collapse',
 'collapsed',
 'collide',
 'colli

In [202]:
# keywordsInput=train[train["keyword"].isna()]
# keywordsInput.shape
# keywordsInput.sample(20)
# unique.tolist()

In [203]:
# def keywordIn(text):
    
#     for token in text.split():
#         if str(token) in unique:
#             return token
#         else:
#             continue
#     return np.nan

In [204]:
# for i in unique:
#     if str(i).startswith("f"):
#         print(i)
# text=keywordsInput["text"][1]
# print(text)
# print(keywordIn(text))

In [205]:
# keywordsInput.head()

In [206]:
# keywordsInput["keyword"]=keywordsInput["text"].apply(keywordIn)
# keywordsInput.isna().sum()

In [207]:
# # keywordsInput.isna().sum()
# keywordsInput=keywordsInput[keywordsInput["keyword"].isna()]
    

In [208]:
# def lastKeyIn(text):
#     doc=nlp(text)
#     words=[]
#     for word in doc:
#         if word.is_stop or word.is_oov or word.is_space or word.is_punct:
#              pass
#         elif word.is_ascii:
#             if word.text in unique or word.lemma_ in unique:
#                 return word
#             words.append(word)
#     return words

In [209]:
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sentiment=SentimentIntensityAnalyzer()
# def lastKeyIn(text):
#     doc=nlp(text)
#     high_neg=0
#     negative_word=None
#     for word in doc:
#         if word.is_stop or word.is_oov or word.is_space or word.is_punct:
#              pass
#         elif word.is_ascii:
#             if word.text in unique or word.lemma_ in unique:
#                 return word
#             neg=sentiment.polarity_scores(word.text)["neg"]
#             if neg>high_neg:
#                   high_neg=neg
#                   negative_word=word
#     if high_neg<0.5:
#          return("Others")
#     return negative_word

In [210]:
# keywordsInput["keyword"]=keywordsInput["text"].str.lower().apply(lastKeyIn)
# keywordsInput["keyword"].value_counts()

In [211]:
# def keywordMaterial(text):
#     high_neg=0
#     negative_word=None
#     for word in text:
#             neg=sentiment.polarity_scores(word)["neg"]
#             if neg>high_neg:
#                   high_neg=neg
#                   negative_word=word
#     if high_neg<0.5:
#           return("Others")
    
#     return negative_word
# def lastKeyIn(text):
#     doc=nlp(text)
#     high_neg=0
#     negative_word=None
#     for word in doc:
#         if word.is_stop or word.is_oov or word.is_space or word.is_punct:
#              pass
#         elif word.is_ascii:
#             if word.text in unique or word.lemma_ in unique:
#                 return word
#             neg=sentiment.polarity_scores(word)["neg"]
#             if neg>high_neg:
#                   high_neg=neg
#                   negative_word=word
#     if high_neg<0.5:
#          return("Others")
#     return negative_word
        